# A Monte Carlo program for the Newman-Moore triangular lattice Ising model

In [1]:
using Random; Random.seed!(0); 
#k = 2
#L = 2^k
L = 4
N = L*L
Spin = fill(1,N);

In [2]:
#This is the data structure that relates up-triangles to spin indices
UpTriangle = zeros(Int,N,3)
for i = 1:N
    UpTriangle[i,1] = i
    UpTriangle[i,2] = i+1    
    UpTriangle[i,3] = i+L
    #fix PBCs
    if mod(i,L) == 0
        UpTriangle[i,2] = i + 1 - L
    end
    if (i+L)> N
        UpTriangle[i,3] = i + L - N
    end
end

In [3]:
#This is the inverse data structure that relates a spin index to its 3 up-triangles
AssociatedTri = zeros(Int,N,3)
for i = 1:N
    AssociatedTri[i,1] = i
    AssociatedTri[i,2] = i-1
    AssociatedTri[i,3] = i-L
    if mod(i-1+L,L) == 0
        AssociatedTri[i,2] = i - 1 + L
    end
    if i<(L+1) 
        AssociatedTri[i,3] = i + N - L
    end
end

In [4]:
#here is the brute force calculation of the energy
function Energy_Total(Energy)
    for i = 1:N
        Spin1 = Spin[UpTriangle[i,1]]
        Spin2 = Spin[UpTriangle[i,2]]
        Spin3 = Spin[UpTriangle[i,3]]
        Energy += 0.5 * Spin1 * Spin2 * Spin3  #J = 1
    end
    return Energy
end #Energy_Total

Energy_Total (generic function with 1 method)

In [5]:
#here is the energy calculated from the local triangular plaquettes
function Energy_Diff(Energy,spin_index)
       
    Tri1 = AssociatedTri[spin_index,1]
    Tri2 = AssociatedTri[spin_index,2]
    Tri3 = AssociatedTri[spin_index,3]

    local_e_before = 0
    Spin1 = Spin[UpTriangle[Tri1,1]]
    Spin2 = Spin[UpTriangle[Tri1,2]]
    Spin3 = Spin[UpTriangle[Tri1,3]]
    local_e_before += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = Spin[UpTriangle[Tri2,1]]
    Spin2 = Spin[UpTriangle[Tri2,2]]
    Spin3 = Spin[UpTriangle[Tri2,3]]
    local_e_before += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = Spin[UpTriangle[Tri3,1]]
    Spin2 = Spin[UpTriangle[Tri3,2]]
    Spin3 = Spin[UpTriangle[Tri3,3]]
    local_e_before += 0.5 * Spin1 * Spin2 * Spin3  

    #flip the spin
    Spin[spin_index] *= -1
    
    local_e_after = 0
    Spin1 = Spin[UpTriangle[Tri1,1]]
    Spin2 = Spin[UpTriangle[Tri1,2]]
    Spin3 = Spin[UpTriangle[Tri1,3]]
    local_e_after += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = Spin[UpTriangle[Tri2,1]]
    Spin2 = Spin[UpTriangle[Tri2,2]]
    Spin3 = Spin[UpTriangle[Tri2,3]]
    local_e_after += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = Spin[UpTriangle[Tri3,1]]
    Spin2 = Spin[UpTriangle[Tri3,2]]
    Spin3 = Spin[UpTriangle[Tri3,3]]
    local_e_after += 0.5 * Spin1 * Spin2 * Spin3  
    
    println(spin_index," ",local_e_before," ",local_e_after)
    
    return Energy + local_e_after - local_e_before
end

Energy_Diff (generic function with 1 method)

In [6]:
Energy = 0
Energy=Energy_Total(Energy)
println(Energy)

8.0


In [7]:
spin_i = 2
println(Energy_Diff(Energy,spin_i)," ")

2 1.5 -1.5
5.0 


In [8]:
spin_i = 2
println(Energy_Diff(Energy,spin_i)," ")

2 -1.5 1.5
11.0 
